<a href="https://colab.research.google.com/github/dowrave/RoadToImageSeg_GAN/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전이 학습(Transfer Learning)
- 사전 훈련된 네트워크에서 전이 학습을 사용하여 이미지 분류에 이용할 수 있다
- 이 방법의 장점으로는, <b> 적은 데이터가 있더라도 이미지 분류를 수행할 수 있다 </b>는 점에 있다.
- [텐서플로우 튜토리얼](https://www.tensorflow.org/tutorials/images/transfer_learning?hl=ko)

### 역시 코드로 써봐야 뭐하는지 알 수 있음


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

## 데이터 전처리

### 이미지 다운로드

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs_zip', 
                                      origin = _URL, # URL의 파일을 다운받음
                                      extract = True) # 압축도 풂
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered') # 경로명 설정. dirname을 이용할 수 있다.

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir, shuffle = True,
                                             batch_size = BATCH_SIZE, image_size = IMG_SIZE) # 데이터셋에 집어넣음

# BATCH SIZE는 보통 모델에 집어넣을 때 정하는 거 아니었나? -> train_dataset은 BatchDataset 이라는 자료 유형을 가짐.

In [ ]:
(160, 160) + (3, )

In [ ]:
validation_dataset = image_dataset_from_directory(validation_dir, shuffle = True, 
                                                  batch_size = BATCH_SIZE, image_size = IMG_SIZE)

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize = (10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_names[labels[i]])
    plt.axis('off')

In [ ]:
# 테스트 세트 만들기
val_batches = tf.data.experimental.cardinality(validation_dataset) # 검증 세트에서 사용할 수 있는 데이터 배치 수 확인, 그 중 20%를 테스트 세트로 이동
test_dataset = validation_dataset.take(val_batches // 5) # 6
validation_dataset = validation_dataset.skip(val_batches // 5) # 26

In [ ]:
print(tf.data.experimental.cardinality(validation_dataset), tf.data.experimental.cardinality(test_dataset))

### 성능을 높이는 데이터세트 구성
- 버퍼링된 프리페치(Prefetch)를 사용하여 I/O 차단 없이 디스크에서 이미지를 드롭한다. (설정이 없다면 파일 읽기 - 불러오기 - 훈련하기 3가지 기능이 동시에 실행되지 않고 하나씩만 실행됨.)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size = AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size = AUTOTUNE)

### 데이터 증강
- 이미지 데이터셋이 크지 않다면, 회전, 뒤집기 등 무작위이지만 사실적인 변환을 적용해 샘플에 다양성을 부여한다.

In [ ]:
data_augmentation = tf.keras.Sequential([
                                         tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                         tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])
# 위 레이어는 model.fit을 호출할 때 훈련 중에만 활성화되며, evaluate 또는 fit의 추론모드 에서 모델을 사용하면 비활성화 된다.

In [ ]:
# 증강 결과 확인
for image, _ in train_dataset.take(1): # 1개의 데이터셋을 불러옴
  plt.figure(figsize = (10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i+1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### 픽셀 값 재조정
- 다운받을 모델은 [-1, 1]의 픽셀 값을 예상한다. 

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# 픽셀값 스케일링
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset = -1)

In [ ]:
# 만약 다른 모델을 사용한다면 해당 API 문서에 가서 입력값으로 뭘 요구하는 지 확인할 것
# 혹은 preprocess_input 함수가 내장되어 있으니 그걸 써도 된다.

In [ ]:
# 사전 훈련된 CNN에서 기본 모델 생성하기
# 불러올 모델은 140만개의 이미지와 1000개의 클래스로 구성된 대규모 데이터셋인 ImageNet을 사용해 훈련된 모델이다.
# 특성 추출에 사용할 MobileNet V2 레이어를 선택해야 한다. 어.. 그냥 코드를 써보고 보자

In [ ]:
IMG_SHAPE = IMG_SIZE + (3, )
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                               include_top = False, # top에는 분류 층이 들어간다. 이를 제외함
                                               weights = 'imagenet')

In [ ]:
# 이 특징 추출기는 160 160 3 -> 5 5 1280으로 변환한다.
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch) 
print(feature_batch.shape)

## 특징 추출

In [ ]:
# Base Model 고정하기 : 주어진 레이어의 가중치 업데이트를 방지하는 코드임

base_model.trainable = False # 전체 모델의 trainable을 false로 가져감

In [ ]:
# 많은 모델의 뒤에는 BatchNormalization 레이어가 포함되어 있다.
# 위에서 trainable = False로 가져갔기 때문에 이는 '추론 모드'에서 실행되며, 평균 & 분산 통계를 업데이트하지 않는다.
base_model.summary()

In [ ]:
# 분류층 추가
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() # 특성을 이미지당 1개의 1280 element를 갖는 벡터로 변환, 5*5 공간 위치 평균을 구한다
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(1) # 이미지당 단일 예측으로 변환. logit 혹은 원시 예측 값으로 취급되므로 활성화함수가 필요 없다.
                                            # 즉 양수는 클래스 1, 음수는 클래스 0.
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
# Keras Functional API를 사용, 데이터 증강, 크기 조정, base_model 및 특성 추출기 레이어를 연결해 모델을 구축한다.
inputs = tf.keras.Input(shape = (160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training = False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# 모델 컴파일
base_learning_rate = 0.0001
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables) # 2개는 가중치, 바이어스

In [ ]:
# 모델 훈련
initial_epochs = 10
loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print(loss0, accuracy0)

In [ ]:
history = model.fit(train_dataset,
                    epochs = initial_epochs,
                    validation_data = validation_dataset)

In [ ]:
# 학습 곡선 (Fixed Feature Extractor)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize = (8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label= ' Training Accuracy')
plt.plot(val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and Validation Accuracy')


plt.subplot(2, 1, 2)
plt.plot(loss, label= ' Training Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, 1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# 미세 조정
# 사전 훈련된 네트워크의 가중치는 학습 중 업데이트되지 않았다.
# 성능 향상 방법 : 추가한 분류기의 훈련과 함께, 사전 훈련된 모델의 최상위 레이어 가중치를 훈련(=미세 조정)하는 것이다.
# 훈련을 통해 가중치는 일반적인 특징 맵에서 개별데이터셋과 관련된 특징으로 조절된다.
# 주의 : 사전 훈련 모델을 훈련 불가능으로 설정할 것. 무작위로 초기화된 분류기를 추가하고 모든 레이어를 훈련하려 한다면 업데이트의 크기가 너무 커짐
      # 대충 사전 훈련이라는 말의 의미가 없어진다는 뜻

In [ ]:
# base_model을 고정 해제 후 맨 아래층을 훈련할 수 없도록 설정하면 된다.
base_model.trainable = True

print('# of Layers in the base model: ', len(base_model.layers))

# fine tune
fine_tune_at = 100

# FREEZE ALL LAYERS BEFORE FINE_TUNE_AT layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
# 사전 훈련된 가중치를 다시 조절하려면 컴파일 단계에서 낮은 학습률을 사용하는 게 중요하다
# 그렇지 않으면 너무 빠르게 과대적합됨
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
               optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate / 10),
               metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
# 모델 훈련 계속하기
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs = total_epochs,
                         initial_epoch = history.epoch[-1], # 이어서 학습하는 건 그냥 여기만 잘 지정해주면 됨
                         validation_data = validation_dataset)

In [ ]:
# 마지막 몇 층을 미세조정, 그 위의 분류기를 훈련할 때의 정확도 / 손실의 학습 곡선 확인
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize = (8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label= ' Training Accuracy')
plt.plot(val_acc, label = 'Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1, initial_epochs-1], 
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc = 'lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')


plt.subplot(2, 1, 2)
plt.plot(loss, label= ' Training Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1, initial_epochs-1], 
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# 성능 평가 및 예측
loss, accuracy = model.evaluate(test_dataset)
print("Test Accuracy : ", accuracy)

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print("Predictions : \n", predictions.numpy())
print('Labels : \n', label_batch)

plt.figure(figsize = (10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i+1)
  plt.imshow(image_batch[i].astype('uint8'))
  plt.title(class_names[predictions[i]])
  plt.axis('off')